# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

# 0) Complete example
See the [sample notebook](/Quickstarts/auto3d_conformer_tautomer_generation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 1.1) Configuration

In [ ]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

2024-03-16 10:23:41,783 - rush - WARNING - Module gmx has a different version on the server: github:talo/tengu-gmx/bf21f5d70350a4276b640ddcf05706c65f5fc926#gmx_tengu.
                                    Use `.update_modules()` to update the lock file
2024-03-16 10:23:41,783 - rush - WARNING - Module gmx_resume has a different version on the server: github:talo/tengu-gmx/bf21f5d70350a4276b640ddcf05706c65f5fc926#gmx_resume_tengu.
                                    Use `.update_modules()` to update the lock file
2024-03-16 10:23:41,783 - rush - WARNING - Module fragment_aa has a different version on the server: github:talo/tengu-prelude/4a16cf12be35f0a40c113d4410046de865f1906f#fragment_aa.
                                    Use `.update_modules()` to update the lock file
2024-03-16 10:23:41,784 - rush - WARNING - Module fragment has a different version on the server: github:talo/tengu-prelude/4a16cf12be35f0a40c113d4410046de865f1906f#fragment.
                                    Use `.up

## 1.2) Build your client

In [ ]:
# Get our client, for calling modules and using the rush API
client = rush.build_blocking_provider_with_functions(batch_tags=TAGS)

# 2) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

auto3d(*args: *tuple[RushObject[bytes], str, Record], **kwargs) -> tuple[RushObject[None], RushObject[list[Record]]]
    Generate 3D conformers from SMILES strings and other inputs

    Module version:
    `github:talo/tengu-auto3d/bd31770c27581753010b6623cdd4bd82b0628e79#auto3d_tengu`

    QDX Type Description:

        molecule_file: Object[@$Bytes];
        molecule_file_type: string;
        options: Auto3dOptions {
            verbose: bool?,
            capacity: u32?,
            k: i32?,
            gpu_idx: [u32]?,
            mpi_np: u32?,
            patience: u32?,
            threshold: f32?,
            convergence_threshold: f32?,
            enumerate_tautomer: bool?,
            memory: u32?,
            max_confs: u32?,
            window: f32?,
            use_gpu: bool?,
            job_name: string?,
            enumerate_isomer: bool?,
            opt_steps: u32?,
            optimizing_engine: Auto3dOptimizingEngi

In [ ]:
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = client.auto3d(
    ligand_path,
    "smi",
    {"k": 5, "use_gpu": True},
    resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
)

print(f"{datetime.now().time()} | Running ligand preparation!")

10:18:31.777693 | Running ligand preparation!


In [ ]:
conformer_qdxf.source

UUID('bab45393-f615-434e-9bea-9ea89e39b34c')

In [ ]:
try:
    conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

2024-03-16 10:18:31,893 - rush - INFO - Argument 4294bc30-c00f-47d3-91b4-36625dd7bd9f is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:18:36,195 - rush - INFO - Argument 4294bc30-c00f-47d3-91b4-36625dd7bd9f is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:22:53,249 - rush - INFO - Argument 4294bc30-c00f-47d3-91b4-36625dd7bd9f is now ModuleInstanceStatus.DISPATCHED
2024-03-16 10:22:54,318 - rush - INFO - Argument 4294bc30-c00f-47d3-91b4-36625dd7bd9f is now ModuleInstanceStatus.RUNNING
2024-03-16 10:23:24,365 - rush - INFO - Argument 4294bc30-c00f-47d3-91b4-36625dd7bd9f is now ModuleInstanceStatus.AWAITING_UPLOAD


You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 3) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

hermes_energy(*args: *tuple[RushObject[Record], Record, Optional[Record], Optional[Record], Optional[Record], Optional[Record]], **kwargs) -> tuple[RushObject[Record]]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided

    Module version:
    `github:talo/tengu-prelude/464d2641a6109f6b831ec0e62a846b9283652d0f#hermes_energy`

    QDX Type Description:

        input: Object[Conformer];
        system: System {
            teams_per_node: u32?,
            max_gpu_memory_mb: u32?,
            gpus_per_team: u32?,
            oversubscribe_gpus: bool?
        };
        model: Model {
            basis: string,
            force_cartesian_basis_sets: bool?,
            aux_basis: string?,
            standard_orientation: string?
        }?;
        scf keywords: SCF {
            convergence_threshold: f64?,
      

In [ ]:
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

In [ ]:
(conformer,) = client.pick_conformer(conformer_qdxf, 0)

In [ ]:
conformer_out = json.load(open(conformer.download(overwrite=True), "r"))

In [ ]:
(hermes_energy,) = client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    None,
    None,
    None,
    target="NIX_SSH",
    resources=HERMES_RESOURCES,
)

In [ ]:
hermes_energy.get()

2024-03-16 10:42:58,293 - rush - INFO - Argument 588a90b6-7930-445d-8208-7ddfa393cb46 is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:43:02,750 - rush - INFO - Argument 588a90b6-7930-445d-8208-7ddfa393cb46 is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:43:06,033 - rush - INFO - Argument 588a90b6-7930-445d-8208-7ddfa393cb46 is now ModuleInstanceStatus.RUNNING
2024-03-16 10:43:17,679 - rush - INFO - Argument 588a90b6-7930-445d-8208-7ddfa393cb46 is now ModuleInstanceStatus.AWAITING_UPLOAD


'https://storage.googleapis.com/qdx-store/01dde466-4ba8-4ea2-b188-712b747bbd93?x-goog-signature=474f1a262bfcc268afd1e99b302321ae810aa990a3a70087a9af6efb3389786acad5f68985b20dc48b37d2f8fed730c0fe26f2658955015cec31ca57e69beb78053afcd78980df0549c89a891f33d0857c132c258c74ba2d7b0cdca64861f4877fca3c74c0df2d97d1e196263afe87041d5974d704e60ed10e46f5e612b95e969ff77a7af42600b67369aef7b65837fb7f789935da32f5eaa36bf6b5c1979e7e4f28c771f33ee34e94347cf67a6627201375dcc139bbc61971b0e51d54450dad55182686bb16243ce47c71d6acbbcc636fa3053ac3ba98ae8314b7b3b3f0648f26a930ef7302d2d3f8bf13f0ec8d1e914ce01807d8227c032bc7c80379589c7b&x-goog-algorithm=GOOG4-RSA-SHA256&x-goog-credential=qdx-store-user%40humming-bird-321603.iam.gserviceaccount.com%2F20240316%2Fasia-southeast1%2Fstorage%2Fgoog4_request&x-goog-date=20240316T024333Z&x-goog-expires=3600&x-goog-signedheaders=host'